In [1]:
from imports.ExtractContactCases import ExtractContactCases

ex = ExtractContactCases('../data/bags/data2', '../data/extractions/contact_extraction4/')
samples = ex.load()

In [47]:
import numpy as np
import torch
import torch_geometric.transforms as T
from torch_geometric.nn.pool import radius_graph, knn_graph
from torch_geometric.data import Data, DataLoader, Dataset

im_height=260
im_width=346

temporal_edge_order = True

sample_id = 'sample_1'

events = np.array(samples[sample_id]['events'])

coord1, coord2 = torch.tensor(events[:, 0:2].astype(np.float32)).T 
ts = events[:, 2]
ts = ((ts - ts.min()) / (ts.max() - ts.min())).astype(np.float32)
coord3 = torch.tensor(ts)
pos = torch.stack((coord1 / im_width, coord2 / im_height, coord3)).T

feature = torch.tensor(events[:, 3].astype(np.float32))
feature = feature.view(-1, 1)

case = samples[sample_id]['case']
edge_index = radius_graph(pos, r=0.05, max_num_neighbors=16)
all_edges = edge_index.shape[1]

if temporal_edge_order:
    row, col = edge_index
    temp_order_edges = pos[row, 2] <= pos[col, 2]
    edge_index = edge_index[:, temp_order_edges]
    print(all_edges - edge_index.shape[1])


#edge_index = knn_graph(pos, knn)
#edge_index, _, mask = remove_isolated_nodes(edge_index=edge_index, num_nodes=feature.shape[0])

y = torch.tensor(np.array(ex.cases_dict[case], dtype=np.float32)).reshape(1, -1)

transforms = T.Compose([T.Cartesian(cat=False, norm=True), T.LargestConnectedComponents(1)])

num_nodes_og = len(feature)
data = Data(x=feature, edge_index=edge_index, pos=pos, y=y)
data = transforms(data)
print(num_nodes_og - data.x.shape[0] )
data

34151
109


Data(x=[4468, 1], edge_index=[2, 33546], y=[1, 2], pos=[4468, 3], edge_attr=[33546, 3])

In [12]:
row, col = data.edge_index

In [18]:
sum(data.pos[row, 2] <= data.pos[col, 2])

tensor(47763)

In [37]:
from torch_geometric.utils import to_scipy_sparse_matrix, get_num_hops, remove_isolated_nodes
from scipy.sparse.csgraph import connected_components, dijkstra

#edge_index_, _, mask = remove_isolated_nodes(data.edge_index)
adj = to_scipy_sparse_matrix(data.edge_index)
n, comps = connected_components(adj)
n

1

In [4]:
np.logical_not(mask)

tensor([1, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8)

In [38]:
np.unique(comps, return_counts=1)

(array([0], dtype=int32), array([4468]))

In [39]:
from models.modules import GraphRes

model = GraphRes()
get_num_hops(model, )

7

In [40]:
dij = dijkstra(adj)

In [48]:
plt.hist(dij[np.logical_and(dij > 0, dij < np.inf)].ravel())

(array([2.21461e+05, 5.39728e+05, 7.40924e+05, 6.37836e+05, 2.85997e+05,
        1.63375e+05, 3.47170e+04, 5.53900e+03, 1.42400e+03, 2.17000e+02]),
 array([ 1. ,  4.8,  8.6, 12.4, 16.2, 20. , 23.8, 27.6, 31.4, 35.2, 39. ]),
 <BarContainer object of 10 artists>)

In [34]:
dij.shape

(15668, 15668)

In [33]:
" "

' '

In [8]:
[(i, data.pos[comps==i].shape[0]) for i in np.unique(comps)]

[(0, 1),
 (1, 2),
 (2, 6384),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 3),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 3),
 (39, 1),
 (40, 2),
 (41, 2),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 4),
 (47, 5),
 (48, 1),
 (49, 1),
 (50, 2),
 (51, 1),
 (52, 4),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 6),
 (57, 1),
 (58, 2),
 (59, 2),
 (60, 1),
 (61, 3),
 (62, 5),
 (63, 1),
 (64, 4),
 (65, 2),
 (66, 1)]

In [9]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib tk

events_data = events
ts = (events[:,2].astype(np.float64) - events[0,2].astype(np.float64))/1e9

events_positive = events_data[events_data[:,3]==1, 0:3]

events_negative = events_data[events_data[:,3]==0, 0:3]

# Visualize events in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
px_pos = events_positive[:,0]
py_pos = events_positive[:,1]
t_st_pos = ts[events_data[:,3]==1]
ax.scatter(px_pos, py_pos, t_st_pos, c="blue", s=5, marker='.')
px_neg = events_negative[:,0]
py_neg = events_negative[:,1]
t_st_neg = ts[events_data[:,3]==0]
ax.scatter(px_neg, py_neg, t_st_neg, c="blue", s=5, marker='.')
ax.scatter(events[np.logical_not(mask), 0], events[np.logical_not(mask), 1], ts[np.logical_not(mask)], c='red', s=5)
plt.show()

In [22]:
np.logical_not(mask).sum()

tensor(52)

In [108]:
(events[:,2].astype(np.float64) - events[0,2].astype(np.float64))/1e9

array([0.        , 0.00061901, 0.00209382, ..., 0.09976704, 0.0997719 ,
       0.09980083])

In [105]:
events[:, 2]

array([1672235585829437634, 1672235585830056634, 1672235585831531634, ...,
       1672235585929204634, 1672235585929209634, 1672235585929238634])

In [111]:
import matplotlib.pyplot as plt
plt.xlim((0,1))
plt.ylim((0,1))

plt.scatter(*data.pos[comps==9, :2].T, c='b')
plt.scatter(*data.pos[comps==27, :2].T, c='r')


IndexError: The shape of the mask [6438] at index 0 does not match the shape of the indexed tensor [6487, 2] at index 0

In [54]:
[(i, data.pos[comps==i].shape[0]) for i in np.unique(comps)]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 4511),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 2),
 (42, 3),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 6),
 (49, 2),
 (50, 2),
 (51, 1),
 (52, 1)]

53

In [22]:
n

53